# Arrays

**Question 5.1**: The Dutch national flag problem

Sort into 3 partitions using a pivot, similar to how the Dutch flag is separated into 3 colors.

*example*:
suppose A = <0, 1, 2, 0, 2, 1, 1> and pivot index is 3.
    A[3] = 0 so <0, 0, 1, 2, 2, 1, 1>
if the pivot index was 2 then,
    A[2] = 2 so <0, 1, 0, 1, 1, 2, 2> or <0, 0, 1, 1, 1, 2, 2>

so basically LESSTHAN before PIVOT DUPES followed by GREATERTHAN.

If we aren't being space conscious, it'd be easy to create 3 new lists that'll be glued together
to make the final partition and then iterate through the given array to populate those three
lists. The time complexity would be O(n) since you'd only need to iterate through the array
once and the space complexity would be O(n) as you'd be making a new array with the same amount
of elements in the original.

To improve upon that is to then shift the elements in the given array into the three desired
partitions. To do that, I think pointers might be best. These pointers would show the boundaries
of each parition: lessthan, pivot (and its dupes), and the greater than. These three pointers
will also wrap around an extra section of unlabeled values. That way as the pointers are iterated
down or up, two pointers will meet when the unknown section is exhausted.
In our case, we'll have the unknown section be between the pivot pointer and the greater than pointer,
which would have us putting the pivot in the beginning of the array to start and then the greater thans
at the end.

In [2]:
def dutch_flag_partition(A: list[int], pivot: int) -> None:
    # specifically, less will be the first occurrence of the pivot
    # greater will be the end of the unknown section
    # pivot will be the first value in the unknown section
    less, greater = 0, len(A)-1
    x = A[pivot]
    A[less], A[pivot] = A[pivot], A[less]
    while greater >= pivot:
        if A[greater] == x:
            A[greater], A[pivot] = A[pivot], A[greater]
            pivot += 1
        elif A[greater] < x:
            A[pivot], A[less] = A[less], A[pivot]
            A[greater], A[less] = A[less], A[greater]
            less, pivot = less + 1, pivot + 1
        else:
            greater -= 1
            
# time complexity: O(n), n being the length of A
# space complexity: O(1)

Book holds a similar answer. The only change is that the unknown variable being switched is the first
element in that section rather than the last, which eliminates one switch in the the unknown being
less than x.

**Question 5.6**: Buy and sell a stock once

To brute force this, you just compare an index with each of the following indicies to find the highest profit for an O(n^2) time complexity.

If we want an O(n) time complexity, you could use a variables to keep track of values as you iterate
through the stock prices.
Because you want a lower buy price than the sell price, the buy price or low number will always be
earlier in the list. Therefore, to find the biggest difference, you keep track of the lowest buy price and
compare with the current index in the stock list. If that index is lower, replace the lowest buy price and continue.
If the profit is greater than previously, replace the profit before continuing.

In [3]:
# assumes that the prices are ints. can easily replace int with float
def buy_and_sell_stock(A: list[int]) -> int:
    profit, low = 0, A[0]
    
    for price in A:
        if price < low:
            low = price
        profit = max(profit, price - low)
    
    return profit

In [1]:
# Book Answer
# uses float instead of int
def buy_and_sell_stock_once(prices: list[float]) -> float:
    min_price_so_far, max_profit = float('inf'), 0.0
    for price in prices:
        # instead of a conditional statement, this just goes ahead
        # and calculates the profit
        max_profit_sell_today = price - min_price_so_far
        max_profit = max(max_profit, max_profit_sell_today)
        min_price_so_far = min(min_price_so_far, price)
    return max_profit

**Question 5.12**: Sample offline data

So you get an array of distinct elements and size and must return a subset of the given size of the array elements.
All subsets should be equally likely, return result in input array itself.

My first thought is using a random generator to pick an index in the array to add to the subset. By placing the
selected value in the front of the array, we can keep track of how many have been picked up and pick from the
remaining indicies until k amount of elements are picked.

In [5]:
import random

def sample_data(A: list[int], k: int) -> None:
    for i in range(k):
        j = randint(i, len(A)-1)
        A[i], A[j] = A[j], A[i]

# time complexity: O(k) since random sort is O(1) and we only need to choose k values
# space complexity: O(1) as we use the original array to store the answer.

**Question 5.18**: Compute the spiral ordering of a 2D array

Write a program that takes an n x n 2D array and returns the spiral ordering of the array.
*hint*: Use case analysis and divide-and-conquer

Look at the diagram (provided in the book) some patterns stand out. Aside from the first line in the spiral, which is always across row0, each sequential line is that row length - 1 twice before decreasing by length again. There's also an alternating pattern of row then col then row then col. Because of these patterns, a recursive solution can be found. A separate list can keep track of each element passed which'll give us the list that'll be returned.

In [7]:
def spiral_ordering(square: list[list[int]]) -> list[int]:
    spiral = []
    
    # the recursive method that'll go along each edge of the spiral
    def edge(row: int, col: int, l: int, direction: int) -> None:
        # direction determines if we're going across a row or col
        #     up or down
        # l is the length of the edge
        # row and col holds the last location placed in spiral
        
        for i in range(l):
            if direction == 0: # down col
                row += 1
            elif direction == 1: # left row
                col -= 1
            elif direction == 2: # up col
                row -= 1
            else: # direction == 3 or right row
                col += 1
            spiral.append(square[row][col])
        
        if direction == 0 or direction == 2:
            l -= 1
        if len(spiral) != len(square)**2: # continue spiraling
            edge(row, col, l, (direction + 1)%4)
    
    l = len(square)
    row, col = 0, -1
    for i in range(l):
        col += 1
        spiral.append(square[row][col])
    edge(row, col, l, 0)
    
    return spiral

# Time Complexity: We go through every element of the array so O(n) of n elements
#                  then we add the amount of recursive calls which is O(2k-1) where
#                  k is the length of the square's row or O(k) simplified.
#                  Together that makes a time of O(n+k)
# Space Complexity: O(n) for the list that holds the spiral sequence of the 2d array.

The book presents two slightly different solutions and also calculates the time complexity as O(n^2) for n being the length of the square's side.
The first solution provided is also a recursive solution but instead of recursiving for every edge, they do it for every outer square not yet added to the list.
The second solution is similar to mine where edges are traversed, but the difference is that they didn't implement recursive calls like I did. Therefore, it has a improved time complexity as the stacked recursive calls aren't added in.

Second solution before for direct comparisons.

In [8]:
def matrix_in_spiral_order(square_matrix: list[list[int]]) -> list[int]:
    # using immutable set to keep track of directions is a nice way of
    # simplifying which way we're traversing the square without a bunch
    # of conditional statements
    shift = ((0, 1), (1, 0), (0, -1), (-1, 0))
    direction = x = y = 0
    spiral_ordering = []
    
    for _ in range(len(square_matrix)**2):
        spiral_ordering.append(square_matrix[x][y])
        square_matrix[x][y] = 0 # I'm not sure why this is added. maybe cleanup?
        next_x, next_y = x + shift[direction][0], y + shift[direction][1]
        if (next_x not in range(len(square_matrix))
               or next_y not in range(len(square_matrix))
               or square_matrix[next_x][next_y] == 0): # oh it's for collision control
            direction = (direction + 1) & 3
            next_x, next_y = x + shift[direction][0], y + shift[direction][1]
        x, y = next_x, next_y
    return spiral_ordering